In [1]:
''' Load the neccessary Library to Run the website and to do the task '''

from selenium import webdriver           # Load the selenium webdriver to access the browser 
import requests                          # To make a call to the internet web address
from bs4 import BeautifulSoup            # Library needed for Webscraping 
import pandas as pd                      # Library to load data into a tables or tabular panels
import time                              # Time Library
#driver = webdriver.Chrome(executable_path="C:\chromedriver.exe")
#website = 'https://www.propertypro.ng/in/abuja' 
#driver.get(website)

In [25]:
def login():
    '''
    This function loads the web browser driver which is Chrome from the path 
    where it is installed on the computer and opens it. This driver works well
    with the selenium library.

    '''
    
    driver = webdriver.Chrome(executable_path="C:\chromedriver.exe")
    return driver

driver = login()

In [43]:
# User_Request = input("What state do you want to consider?: ")
# state = User_Request.casefold()

In [3]:
# website ='https://www.propertypro.ng/wp-login.php'
# #website = 'https://www.propertypro.ng/in/'+ state +'?page='+str(1)

In [36]:
# select_abuja= driver.find_element_by_link_text(state.title())
# select_abuja.click()

In [91]:
''' 
This function accepts user input of states to look for, then if the state is present in the database,
it gives the state. while it gives error when it is not a part of the database.

'''

User_Request = input("What state do you want to consider?: ")            # ASk for the user choice among the 36 states of Nigeria and the FCT
state = User_Request.casefold()                                          # Convert user response to smaller case so as to ensure no Case error in input

def user_input(state: str):
    if state == 'crossriver':                                            # Since Cross river state is two word unlike the other states, give the reponse in another format based on the website code structure.
        return("Hey user, that is a wrong entry why not put space between or add another state")
            
    elif state == 'cross river':                                          # If the user input is not part of the 36 states, return error
        state = 'cross-river'
    else:
        state

    return state

In [92]:
state_info = user_input(state)

'abuja'

In [76]:
storage = [] 

def get_info(driver,state_info, start: int, stop: int, step: int):
    
    '''   
    Get the website using the driver and selinium library then find the data 
    available in the elements paths, class or selector of the website raw code 
    file.
    Then extract the property typw, location of property, price of property and
    other feature of the property.
    This function iterates to give result of times the user gives.
    '''



    for page in range(start,stop,step):

        website = 'https://www.propertypro.ng/in/'+ state_info +'?page='+str(page) 
        driver.get(website)
        property_type = driver.find_elements_by_css_selector(".listings-property-title")
        location = driver.find_elements_by_xpath('//div[2]/h4')
        price = driver.find_elements_by_xpath("//h3//span[2]")
        
        features = driver.find_elements_by_class_name('fur-areea')
        for i in range(len(property_type)):
            
            print(property_type[i].text)
            print(location[i].text)
            print(price[i].text)
            print(features[i].text)
            storage.append([property_type[i].text, location[i].text, price[i].text, features[i].text])
            time.sleep(1)
    return storage

In [77]:
site_data = get_info(driver,state_info,0,1,1)

3 BEDROOM HOUSE FOR RENT
Zurmi Zamfara
950/MONTH
3 beds 1 baths 0 Toilets
WAREHOUSE FOR SALE
Gusau Zamfara
135,000
beds baths Toilets
COMMERCIAL LAND LAND FOR SALE
Keffi Road Gusau Zamfara
1,500,000
0 beds 0 baths 0 Toilets


In [78]:
def statistics_to_pandas(site_data: list):
    '''  
    Load the data generated from the scraping into a data frame and clean the data to desired taste/result
    
    '''
    df = pd.DataFrame(site_data, columns=["Type of House","Location", "Price", "Features"])  # Create a dataframe using the scrapped data  

    df['Purpose'] = df['Type of House'].str.split(" ").map(lambda x: x[-1])                  # Determine the what the property owner wants either to sell or Rent the property
    df['Type of House'] = df['Type of House'].str.split("FOR").map(lambda x: x[0])           # Get the type of property being considered
    
    return(df)

In [79]:
statistics_to_pandas(site_data)   # Check the Data frame to see Result and make changes where needed

,Type of House,Location,Price,Features,Purpose
0,3 BEDROOM HOUSE,Zurmi Zamfara,950/MONTH,3 beds 1 baths 0 Toilets,RENT
1,WAREHOUSE,Gusau Zamfara,"135,000",beds baths Toilets,SALE
2,COMMERCIAL LAND LAND,Keffi Road Gusau Zamfara,"1,500,000",0 beds 0 baths 0 Toilets,SALE


In [80]:
driver.quit()    #Quit the browser at the end of the scraping.